In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [6]:
DIR = os.getcwd() + "/nfl_predictor"
SCORE_DIR = DIR + "/data/scores"

In [7]:
box_scores = os.listdir(SCORE_DIR)

In [9]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".htm")]

In [10]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    # select what we want to remove 
    # tr is table row and ove_header is the class
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup
    

In [122]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"class": "linescore nohover stats_table no_freeze"})[0]
    line_score = line_score.drop('Unnamed: 0', axis=1)
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score
    

In [123]:
def read_stats(soup):
    df = pd.read_html(str(soup), attrs={"id":"player_offense"}, index_col=0)[0]
    return df

In [139]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])
    # #select all links and re assign names to abbreviations
    nav = soup.select(".linescore_wrap")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    links = [l for l in hrefs if l and "teams" in l and ".htm" in l]
    #set home and away team
    line_score.loc[0, "team"] = links[0].split("/")[2]
    line_score.loc[1, "team"] = links[1].split("/")[2]

    summaries = []
    df = pd.DataFrame(read_stats(soup))

    unique_teams = df['Tm'].unique()
    for team in unique_teams:
        stats = df[df['Tm'] == team]
        stats = stats.apply(pd.to_numeric, errors="coerce")
        
        totals = stats.sum()[1:]
        totals.index = totals.index.str.lower()

        
        maxes = stats.max()[1:]
        maxes.index = maxes.index.str.lower() + "_max"
        
        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            
        summary = summary[base_cols]
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]

    full_game["season"] = links[0].split(".")[0][-4:]

    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games) / len(box_scores)}")

/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore nohover stats_table no_freeze"})[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/284119438.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id":"player_offense"}, index_col=0)[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore no

0.045662100456621


/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore nohover stats_table no_freeze"})[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/284119438.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id":"player_offense"}, index_col=0)[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore no

0.091324200913242


/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore nohover stats_table no_freeze"})[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/284119438.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id":"player_offense"}, index_col=0)[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore no

...

/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore nohover stats_table no_freeze"})[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/284119438.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id":"player_offense"}, index_col=0)[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore no

0.91324200913242


/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore nohover stats_table no_freeze"})[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/284119438.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id":"player_offense"}, index_col=0)[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore no

0.958904109589041


/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore nohover stats_table no_freeze"})[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/284119438.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id":"player_offense"}, index_col=0)[0]
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_22109/3194258525.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"class": "linescore no

In [140]:
games_df = pd.concat(games, ignore_index=True)

In [ ]:
[g.shape[1] for g in games if g.shape[1] != 90]
#games_df.to_csv("nfl_games.csv")

[90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,


In [141]:
games_df

,cmp,att,yds,td,int,sk,yds.1,lng,rate,att.1,...,td.2_max_opp,lng.2_max_opp,fmb_max_opp,fl_max_opp,team_opp,total_opp,home_opp,date,won,season
0,21.0,34.0,194.0,2.0,2.0,0.0,0.0,38.0,72.4,28.0,...,1.0,32.0,0.0,0.0,sfo,24,1,2024-01-20,False,2023
1,23.0,39.0,252.0,1.0,0.0,1.0,7.0,32.0,86.7,24.0,...,1.0,38.0,1.0,0.0,gnb,21,0,2024-01-20,True,2023
2,26.0,37.0,297.0,1.0,1.0,2.0,21.0,39.0,91.8,18.0,...,1.0,24.0,1.0,0.0,phi,25,1,2018-11-25,False,2018
3,20.0,28.0,236.0,1.0,0.0,3.0,22.0,24.0,108.6,29.0,...,1.0,39.0,1.0,0.0,nyg,22,0,2018-11-25,True,2018
4,15.0,31.0,169.0,0.0,2.0,2.0,8.0,25.0,38.2,30.0,...,0.0,29.0,1.0,0.0,cle,21,1,2018-09-20,False,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,30.0,40.0,446.0,4.0,0.0,4.0,31.0,72.0,187.6,19.0,...,1.0,53.0,0.0,0.0,kan,31,0,2022-01-02,True,2021
4376,24.0,41.0,196.0,0.0,1.0,2.0,21.0,18.0,60.6,27.0,...,0.0,27.0,0.0,0.0,nwe,27,1,2016-09-22,False,2016
4377,11.0,19.0,103.0,0.0,0.0,1.0,6.0,27.0,72.9,39.0,...,0.0,18.0,0.0,0.0,htx,0,0,2016-09-22,True,2016
4378,21.0,38.0,263.0,1.0,0.0,5.0,34.0,44.0,85.7,16.0,...,1.0,27.0,0.0,0.0,cin,30,1,2018-12-16,False,2018
